In [5]:
import os
import kaggle
import pandas as pd

# Configurar las credenciales de Kaggle
os.environ['KAGGLE_USERNAME'] = 'sergiogrigorow'
os.environ['KAGGLE_KEY'] = '53eaa416645f87b31ba66153190551e0'

# Autenticarse en Kaggle
kaggle.api.authenticate()

# Ruta donde se guardará el archivo descargado
directorio = 'C:/Users/sergi/Proyectos/Proyecto_Final/data/raw'

# Descargar y descomprimir el dataset
kaggle.api.dataset_download_files('austinreese/craigslist-carstrucks-data', path=directorio, unzip=True)



Dataset URL: https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data
           id                                                url  \
0  7222695916  https://prescott.craigslist.org/cto/d/prescott...   
1  7218891961  https://fayar.craigslist.org/ctd/d/bentonville...   
2  7221797935  https://keys.craigslist.org/cto/d/summerland-k...   
3  7222270760  https://worcester.craigslist.org/cto/d/west-br...   
4  7210384030  https://greensboro.craigslist.org/cto/d/trinit...   

                   region                         region_url  price  year  \
0                prescott    https://prescott.craigslist.org   6000   NaN   
1            fayetteville       https://fayar.craigslist.org  11900   NaN   
2            florida keys        https://keys.craigslist.org  21000   NaN   
3  worcester / central MA   https://worcester.craigslist.org   1500   NaN   
4              greensboro  https://greensboro.craigslist.org   4900   NaN   

  manufacturer model condition cylinders  ..

In [52]:
# Leer el archivo CSV descargado
directorio_csv = os.path.join(directorio, 'vehicles.csv')
df = pd.read_csv(directorio_csv)

# Mostrar las primeras filas del DataFrame
print(df.head())

           id                                                url  \
0  7222695916  https://prescott.craigslist.org/cto/d/prescott...   
1  7218891961  https://fayar.craigslist.org/ctd/d/bentonville...   
2  7221797935  https://keys.craigslist.org/cto/d/summerland-k...   
3  7222270760  https://worcester.craigslist.org/cto/d/west-br...   
4  7210384030  https://greensboro.craigslist.org/cto/d/trinit...   

                   region                         region_url  price  year  \
0                prescott    https://prescott.craigslist.org   6000   NaN   
1            fayetteville       https://fayar.craigslist.org  11900   NaN   
2            florida keys        https://keys.craigslist.org  21000   NaN   
3  worcester / central MA   https://worcester.craigslist.org   1500   NaN   
4              greensboro  https://greensboro.craigslist.org   4900   NaN   

  manufacturer model condition cylinders  ... size  type paint_color  \
0          NaN   NaN       NaN       NaN  ...  NaN   NaN

In [53]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

In [54]:
nulos = df.isnull().sum()
nulos

id                   0
url                  0
region               0
region_url           0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
VIN             161042
drive           130567
size            306361
type             92858
paint_color     130203
image_url           68
description         70
county          426880
state                0
lat               6549
long              6549
posting_date        68
dtype: int64

In [55]:
df.describe()

,id,price,year,odometer,county,lat,long
count,4.268800e+05,4.268800e+05,425675.000000,4.224800e+05,0.0,420331.000000,420331.000000
mean,7.311487e+09,7.519903e+04,2011.235191,9.804333e+04,NaN,38.493940,-94.748599
std,4.473170e+06,1.218228e+07,9.452120,2.138815e+05,NaN,5.841533,18.365462
min,7.207408e+09,0.000000e+00,1900.000000,0.000000e+00,NaN,-84.122245,-159.827728
25%,7.308143e+09,5.900000e+03,2008.000000,3.770400e+04,NaN,34.601900,-111.939847
50%,7.312621e+09,1.395000e+04,2013.000000,8.554800e+04,NaN,39.150100,-88.432600
75%,7.315254e+09,2.648575e+04,2017.000000,1.335425e+05,NaN,42.398900,-80.832039
max,7.317101e+09,3.736929e+09,2022.000000,1.000000e+07,NaN,82.390818,173.885502


In [56]:
df = df.drop_duplicates()

In [57]:
df.shape

(426880, 26)

In [35]:
df['VIN']

0                       NaN
1                       NaN
2                       NaN
3                       NaN
4                       NaN
                ...        
426875    1N4AA6AV6KC367801
426876    7JR102FKXLG042696
426877    1GYFZFR46LF088296
426878    58ABK1GG4JU103853
426879    WBA4J1C58KBM14708
Name: VIN, Length: 426880, dtype: object

In [38]:
df['region_url']

0           https://prescott.craigslist.org
1              https://fayar.craigslist.org
2               https://keys.craigslist.org
3          https://worcester.craigslist.org
4         https://greensboro.craigslist.org
                        ...                
426875       https://wyoming.craigslist.org
426876       https://wyoming.craigslist.org
426877       https://wyoming.craigslist.org
426878       https://wyoming.craigslist.org
426879       https://wyoming.craigslist.org
Name: region_url, Length: 426880, dtype: object

In [58]:
columnas_eliminar = ["county", "VIN", "region_url", "image_url", "description", "url"]
df = df.drop(columns = columnas_eliminar)

In [42]:
df.shape

(426880, 20)

In [59]:
nulos = df.isnull().sum()
nulos

id                   0
region               0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
drive           130567
size            306361
type             92858
paint_color     130203
state                0
lat               6549
long              6549
posting_date        68
dtype: int64

In [60]:
del_nulos = ['year','model','odometer','title_status','transmission','lat','long']
df = df.dropna(subset= del_nulos )

In [61]:
nulos = df.isnull().sum()
nulos

id                   0
region               0
price                0
year                 0
manufacturer     15832
model                0
condition       158721
cylinders       168477
fuel              1554
odometer             0
title_status         0
transmission         0
drive           121168
size            287711
type             88682
paint_color     118009
state                0
lat                  0
long                 0
posting_date         0
dtype: int64

In [49]:
df.shape

(400831, 20)

In [62]:
# Crear un diccionario para mapear cada modelo a su fabricante más frecuente
model_to_manufacturer = df.groupby('model')['manufacturer'].apply(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown')

# Función para rellenar los valores nulos de manufacturer
def fill_manufacturer(row):
    if pd.isnull(row['manufacturer']):
        return model_to_manufacturer.get(row['model'], None)
    return row['manufacturer']

# Aplicar la función al DataFrame
df['manufacturer'] = df.apply(fill_manufacturer, axis=1)


In [64]:
nulos = df.isnull().sum()
nulos

id                   0
region               0
price                0
year                 0
manufacturer         0
model                0
condition       158721
cylinders       168477
fuel              1554
odometer             0
title_status         0
transmission         0
drive           121168
size            287711
type             88682
paint_color     118009
state                0
lat                  0
long                 0
posting_date         0
dtype: int64

In [67]:
df['type']

27           pickup
28           pickup
29           pickup
30           pickup
31            truck
            ...    
426875        sedan
426876        sedan
426877    hatchback
426878        sedan
426879        coupe
Name: type, Length: 400831, dtype: object

In [68]:
# Crear un diccionario para mapear las combinaciones de model, manufacturer, y transmission al type más frecuente
combination_to_type = df.groupby(['model', 'manufacturer', 'transmission'])['type'].apply(lambda x: x.mode()[0] if not x.mode().empty else None).to_dict()

# Función para rellenar los valores nulos de type
def fill_type(row):
    if pd.isnull(row['type']):
        return combination_to_type.get((row['model'], row['manufacturer'], row['transmission']), None)
    return row['type']

# Aplicar la función al DataFrame
df['type'] = df.apply(fill_type, axis=1)

# Reemplazar los valores NaN por cadenas vacías
df['type'] = df['type'].fillna('')



In [70]:
nulos = df.isnull().sum()
nulos

id                   0
region               0
price                0
year                 0
manufacturer         0
model                0
condition       158721
cylinders       168477
fuel              1554
odometer             0
title_status         0
transmission         0
drive           121168
size            287711
type                 0
paint_color     118009
state                0
lat                  0
long                 0
posting_date         0
dtype: int64

In [71]:
combination_to_fuel = df.groupby(['model', 'manufacturer', 'transmission','type'])['fuel'].apply(lambda x: x.mode()[0] if not x.mode().empty else None).to_dict()

def fill_fuel(row):
    if pd.isnull(row['fuel']):
        return combination_to_type.get((row['model'], row['manufacturer'], row['transmission'], row['type']), None)
    return row['fuel']

df['fuel'] = df.apply(fill_fuel, axis=1)

df['fuel'] = df['fuel'].fillna('')

In [72]:
nulos = df.isnull().sum()
nulos

id                   0
region               0
price                0
year                 0
manufacturer         0
model                0
condition       158721
cylinders       168477
fuel                 0
odometer             0
title_status         0
transmission         0
drive           121168
size            287711
type                 0
paint_color     118009
state                0
lat                  0
long                 0
posting_date         0
dtype: int64